# 19.  Tensorboard 可视化好帮手

In [1]:
import tensorflow as tf

## 定义层函数

In [2]:
# 新的Layer定义
def add_layer(inputs, in_size, out_size, activation_function=None):
    """ 添加一个或多个层，并返回该层的输出 """
    with tf.name_scope('layer'):  # 添加范围名称
        with tf.name_scope('weights'):
            Weights= tf.Variable(tf.random_normal([in_size, out_size]), name='W')  # 权重 为输入和输出之间的随机数
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')  # 偏置
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)  # 公式 y = w * x + b
        if activation_function is None:
            outputs = Wx_plus_b  # 没有激活函数就直接输出，线性
        else:
            outputs = activation_function(Wx_plus_b, )  # 套用激活函数，非线性
        return outputs

## 输入层，占位符

In [3]:
# 作为一个整体定义
with tf.name_scope('inputs'):
    # 定义placeholder用于网络输入
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')  # n行1列的数据
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

## 隐藏层

In [4]:
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)  # 每次输入1个神经元，中间输出10个神经元，激励函数是relu

## 输出层，预测值

In [5]:
prediction = add_layer(l1, 10, 1, activation_function=None)  # 输出预测值

## 损失函数

In [6]:
# 预测值和真值之间的误差
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1]))  # 平均，求和，差平方
    # reduction_indices=[1] ??

https://www.zhihu.com/question/51325408?from=profile_question_card

reduce_sum() 就是求和，由于求和的对象是tensor，所以是沿着tensor的某些维度求和。

reduction_indices是指沿tensor的哪些维度求和。

主要是因为对tensor缺乏认识，没有从维度的角度上去思考。

在tensorflow 1.0版本中，reduction_indices被改为了axis。

reduce_xxx这类操作的本质就是降维，以xxx的手段降维。
在所有reduce_xxx系列操作中，都有reduction_indices这个参数，
即沿某个方向，使用xxx方法，对input_tensor进行降维。

reduction_indices的默认值是None，即把input_tensor降到 0维，也就是一个数。
对于2维input_tensor，reduction_indices=0时，按列；reduction_indices=1时，按行。

## 优化器

In [7]:
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)  # 梯度下降法，最小化损失，学习率0.1

## 初始化

In [8]:
init = tf.global_variables_initializer()

## 会话，这里只看图，不训练

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("logs/", sess.graph)

## 终端调用

In [ ]:
!tensorboard --logdir logs

TensorBoard 0.1.6 at http://ubuntu-SSD64G:6006 (Press CTRL+C to quit)


## [程序源码](https://github.com/MorvanZhou/tutorials/blob/master/tensorflowTUT/tf14_tensorboard/full_code.py)

In [ ]:
"""
# View more python learning tutorial on my Youtube and Youku channel!!!

# Youtube video tutorial: https://www.youtube.com/channel/UCdyjiB5H8Pu7aDTNVXTTpcgm
# Youku video tutorial: http://i.youku.com/pythontutorial


from __future__ import print_function
import tensorflow as tf


def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        return outputs


# define placeholder for inputs to network
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

# add hidden layer
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)
# add output layer
prediction = add_layer(l1, 10, 1, activation_function=None)

# the error between prediciton and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

sess = tf.Session()

# tf.train.SummaryWriter soon be deprecated, use following
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:  # tensorflow version < 0.12
    writer = tf.train.SummaryWriter('logs/', sess.graph)
else: # tensorflow version >= 0.12
    writer = tf.summary.FileWriter("logs/", sess.graph)

# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

# direct to the local dir and run this in terminal:
# $ tensorboard --logdir=logs
"""